In [ ]:
import os
import cv2
import torch
import pandas as pd
import numpy as np
from torchvision import models, transforms
from torchvision.models import ResNet50_Weights
from PIL import Image
import unicodedata

# Verificar si hay una GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo pre-entrenado ResNet50 con los pesos más recientes
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1).to(device)
model.eval()  # Poner el modelo en modo evaluación

# Cargar las etiquetas de ImageNet
with open(r'C:\Users\Usuario\Desktop\GitHub\Grupo-Atrium\Deep Learning\Notebook\clases.txt') as f:
    labels = [line.strip() for line in f.readlines()]

# Normalizar rutas para manejar caracteres especiales
def normalize_path(path):
    return unicodedata.normalize('NFKD', path).encode('ascii', 'ignore').decode('ascii')

# Preprocesamiento de imágenes para el modelo
def preprocess_frame(frame):
    """
    Preprocesa un fotograma para que sea compatible con el modelo ResNet50.
    """
    frame = cv2.resize(frame, (224, 224))  # Redimensionar a 224x224
    frame = frame / 255.0  # Normalizar a [0, 1]
    frame = (frame - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]  # Normalizar con mean y std de ImageNet
    frame = torch.tensor(frame, dtype=torch.float32).permute(2, 0, 1)  # Convertir a tensor y cambiar a formato CHW
    return frame.unsqueeze(0).to(device)  # Añadir dimensión del batch y mover a GPU

def detect_objects_in_frame(frame):
    """
    Detecta objetos en un fotograma utilizando el modelo ResNet50.
    Devuelve la clase más probable y la confianza.
    """
    with torch.no_grad():  # Desactivar el cálculo de gradientes
        outputs = model(frame)
    _, predicted = torch.max(outputs, 1)
    confidence = torch.nn.functional.softmax(outputs, dim=1)[0] * 100
    class_name = labels[predicted.item()]
    confidence = confidence[predicted.item()].item()
    print(f"Predicción: {class_name} (Confianza: {confidence:.2f}%)")  # Depuración
    return class_name, confidence

def process_video_frames(frames_dir, batch_size=16):
    """
    Procesa todos los fotogramas de un vídeo y devuelve un DataFrame con los resultados.
    """
    results = []
    frame_files = [os.path.join(root, normalize_path(f)) for root, _, files in os.walk(frames_dir) for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    # Verificar si los archivos existen
    for frame_file in frame_files:
        if not os.path.exists(frame_file):
            print(f"El archivo no existe: {frame_file}")
            continue

        # Cargar la imagen con OpenCV
        frame = cv2.imread(frame_file)
        if frame is None:
            print(f"No se pudo cargar la imagen: {frame_file}")
            continue

        # Preprocesar el fotograma y detectar objetos
        preprocessed_frame = preprocess_frame(frame)
        class_name, confidence = detect_objects_in_frame(preprocessed_frame)
        
        # Guardar los resultados
        results.append({
            'frame_path': frame_file,
            'class_name': class_name,
            'confidence': confidence
        })
    
    return pd.DataFrame(results)

# Ejemplo de uso
if __name__ == "__main__":
    # Directorio que contiene los fotogramas del vídeo
    frames_directory = r"C:\Users\Usuario\Desktop\GitHub\Grupo-Atrium\Deep Learning\frames"
    
    # Procesar los fotogramas y obtener el DataFrame
    results_df = process_video_frames(frames_directory, batch_size=16)  # Ajusta el tamaño del lote según tu GPU
    
    # Guardar los resultados en un archivo CSV
    results_df.to_csv(r'C:\Users\Usuario\Desktop\GitHub\Grupo-Atrium\Deep Learning\Notebook\resultados_deteccion_pytorch.csv', index=False)
    
    print("Detección de objetos completada. Resultados guardados en 'resultados_deteccion_pytorch.csv'.")